In [44]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score

### 데이터 수집

In [6]:
a = pd.read_csv("review.csv")
a

,날짜,제목
0,2024-10-30,악마 기업 [1]
1,2024-10-30,오늘 사면 배당주나요? [2]
2,2024-10-30,거래량의 50%가 프로그램이니
3,2024-10-30,대한독립만세
4,2024-10-30,역시 신한은 다르네 ㅋㅋ
...,...,...
1993,2024-03-20,역사점 신고점 언저리 물리면..ㅜㅜ .(... [1]
1994,2024-03-20,日시중은행 일제히 금리인상 '단숨에 20...
1995,2024-03-20,미-금리 인하 상반기 어려워!!
1996,2024-03-20,어르신들만~참고요!!


In [9]:
b = fdr.DataReader("NAVER : 055550").reset_index() 
b['Date'] = pd.to_datetime(b['Date'])
b['Target'] = np.where(b['Change'] > 0,1,0)
b['Target'].shift(-1)
b = b.dropna()
b

,Date,Open,High,Low,Close,Volume,Change,Target
1,2001-09-11,11037,11277,10894,11181,1239210,0.026345,1
2,2001-09-12,9549,10509,9549,9694,1468150,-0.132993,0
3,2001-09-13,10173,10749,10029,10366,1916800,0.069321,1
4,2001-09-14,10077,10317,9549,9742,1323690,-0.060197,0
5,2001-09-17,9741,9789,9501,9550,620390,-0.019708,0
...,...,...,...,...,...,...,...,...
5709,2024-10-24,56600,56700,55500,56100,613508,-0.003552,0
5710,2024-10-25,56500,59900,56500,58000,2223554,0.033868,1
5711,2024-10-28,57400,57600,55500,56000,1428755,-0.034483,0
5712,2024-10-29,55900,56900,55600,56700,1236712,0.012500,1


In [10]:
a['날짜'] = pd.to_datetime(a['날짜'])

In [12]:
df = a.merge(b, left_on = '날짜', right_on = 'Date')
del df['날짜']
df

,제목,Date,Open,High,Low,Close,Volume,Change,Target
0,악마 기업 [1],2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0
1,오늘 사면 배당주나요? [2],2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0
2,거래량의 50%가 프로그램이니,2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0
3,대한독립만세,2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0
4,역시 신한은 다르네 ㅋㅋ,2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0
...,...,...,...,...,...,...,...,...,...
1732,역사점 신고점 언저리 물리면..ㅜㅜ .(... [1],2024-03-20,48600,49850,48600,48800,1218353,0.019854,1
1733,日시중은행 일제히 금리인상 '단숨에 20...,2024-03-20,48600,49850,48600,48800,1218353,0.019854,1
1734,미-금리 인하 상반기 어려워!!,2024-03-20,48600,49850,48600,48800,1218353,0.019854,1
1735,어르신들만~참고요!!,2024-03-20,48600,49850,48600,48800,1218353,0.019854,1


### 데이터 전처리

In [14]:
from konlpy.tag import Okt

def make_nouns(x):
    return " ".join(okt.nouns(x))

okt = Okt()
df['Nouns'] = df['제목'].apply(make_nouns)
df

,제목,Date,Open,High,Low,Close,Volume,Change,Target,Nouns
0,악마 기업 [1],2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0,악마 기업
1,오늘 사면 배당주나요? [2],2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0,오늘 사면 배당
2,거래량의 50%가 프로그램이니,2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0,거래량 프로그램
3,대한독립만세,2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0,독립 만세
4,역시 신한은 다르네 ㅋㅋ,2024-10-30,55500,55600,54000,54400,1399179,-0.040564,0,역시 신한
...,...,...,...,...,...,...,...,...,...,...
1732,역사점 신고점 언저리 물리면..ㅜㅜ .(... [1],2024-03-20,48600,49850,48600,48800,1218353,0.019854,1,역사 점 신고 점 저리 물리
1733,日시중은행 일제히 금리인상 '단숨에 20...,2024-03-20,48600,49850,48600,48800,1218353,0.019854,1,시중 은행 일제 금리 인상 단숨
1734,미-금리 인하 상반기 어려워!!,2024-03-20,48600,49850,48600,48800,1218353,0.019854,1,금리 인하 상반기
1735,어르신들만~참고요!!,2024-03-20,48600,49850,48600,48800,1218353,0.019854,1,어르신 참고


### 회귀모델 구축

피처

In [36]:
df['Moving Average 10'] = df['Close'].rolling(window=10).mean() 
df['Volatility'] = df['Change'].rolling(window=10).std()  

In [37]:
keywords = ["상승", "하락", "프로그램", "배당"]
vectorizer = CountVectorizer(vocabulary=keywords)
X = vectorizer.fit_transform(df['Nouns'].fillna("")) 
keyword_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df = pd.concat([df, keyword_counts], axis=1)

In [48]:
#감성점수
knu = pd.read_csv("한국어 감성점수 파일.csv")

knu_dic = dict(zip(knu['word'], knu['polarity']))

def sentiment_analysis(text):
    score = 0
    for key, value in knu_dic.items():
        if key in text:
            score += value * text.count(key) 
    return score

df['Sentiment'] = df['Nouns'].apply(sentiment_analysis)

In [49]:
cv = CountVectorizer()
cv_matrix = cv.fit_transform(df['Nouns'])

x = cv_matrix.toarray()
y = df['Target'].values

In [50]:
train_x, test_x, train_y, test_y = train_test_split(x, y) 

model = RandomForestRegressor()
model.fit(train_x, train_y)

RandomForestRegressor()

### 모델 평가

In [51]:
pred = model.predict(test_x)

mse = mean_squared_error(test_y, pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_y, pred)

scores = cross_val_score(model, x, y, scoring='neg_mean_squared_error', cv=5)
rmse_scores = np.sqrt(-scores)

In [53]:
print(rmse, mae)

0.529276308811622 0.4752614411066008


In [52]:
print(rmse_scores, rmse_scores.mean(), rmse_scores.std())

[0.53712615 0.54356166 0.56749918 0.53529842 0.57760563] 0.5522182057905319 0.01712900969013496
